In [1]:
# 1) Mean/Median Age (Age_Days) - Descriptive Statistics Mean/IQR
# 2) Mean/Median Weight (WT_KGS) - Descriptive Statistics Mean/IQR
# 3) Gender distribution (GENDER) - significant differences? - One Way Chi Squre
# 4) Most common Breeds (BREED) - One Way Chi Squre
# 5) Most common shunts (SHUNT) - One Way Chi Squre
# 6) Most common portal vein origin (PORTAL_V) - One Way Chi Squre
# 7) Most common systemic vein insertion (SYSTEM_V) - One Way Chi Squre
# 8) Most common contributing vessels (CONTRB_V) - One Way Chi Squre
# 9) PP score distribution (PP_Score) - One Way Chi Squre and Descriptive Statistics Mean/IQR (age, wt, and pps_n are set as continuous exploratory)

# Cross tab frequencies and ANOVA
# HOSPITAL COMPARISONS (HOSPITAL):
# 10) Do the above values differ AMONG HOSPITALS (HOSPITAL) - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)

# Cross tab frequencies and ANOVA
# COUNTRY COMPARISONS (COUNTRY):
# 11) Do the above values differ AMONG COUNTRIES (COUNTRY)  - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)

# Cross tab frequencies and ANOVA
# BREED COMPARISONS (BREED): [5 MOST COMMON BREEDS]
# 12) What is Gender distribution (GENDER) comparison among 5 most common breeds (BREED)? - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)
# 13) What is Age distribution (Age_Days) comparison among 5 most common breeds (BREED)? - Sub ANOVA limited to breeds 61, 38, 53, 35, 47
# 14) What are 5 most common Shunts  (SHUNT) comparison among 5 most common breeds (BREED)? - Two Way Chi Squre (Comprehensive pairwise comparisons reveals)
# 15) What is PP Score (PP_Score) comparison among 5 most common breeds (BREED)? # Probably best looked at categorically with two-way
# comprehensive chi-square for contingency table first time (Comprehensive pairwise comparisons reveals)


# Regression and ANOVA
# AGE COMPARISONS (Age_Days):
# 16) Does AGE (Age_Days) correlate with Breed (BREED) - RQ 13 handles this with age as DV and Breed (Top 5) as IV with 5 class levels fo breed
# 17) Does AGE (Age_Days) correlate with PP Score (PP_Score) - Standard ANOVA but similar issue with shunt type having a lot of cats, see 18
# 18) Does AGE (Age_Days) correlate with Shunt (SHUNT) - There are 44 different shunt types (shunt type is not ordinal) and age is continuous. 
#  An ANOVA would 
# not yield meaningful results iwth the 44 goups giving a mean age for pairise comparisons.  We could contribute collapsing shunt and 
# bucketing age.


# SHUNT COMPARISONS (SHUNT) ANOVA:
# 19) Does Shunt (SHUNT) correlate with Weight (WT_KGS)    [Above and below mean/median?] - see 20
# 20) Does Shunt (SHUNT) correlate with PP Score (PP_Score) There are 44 different shunt types (non-ordinal) and 11 (with one missing) pp Score.  
# This could be a 
# either a huge cross tab or a difficlt it interpret ANOVA.   Need to define DV/IV an consider collpasing (top 5?) Shunt Type. See 17 and 18

# Breed ANOVA and Cross Tab
# These (21, 22, 23) are all caegorical with multiple and in most cases non-ordinal categorization making inelgible for regression and 
# multiple categories difficult to interpret with thin data classes for ANOVA.  Suggest first round (Comprehensive pairwise comparisons reveals)
# 21) Does PV origin (PORTAL_V) correlate with breed (BREED) # Cross Tab Frequency - Two Way Chi Square
# 22) Does systemic v insertion (SYSTEM_V) correlate with breed (BREED) # Cross Tab Frequency - Two Way Chi Square
# 23) Do contributing vessels (CONTRB_V) correlate with breed (BREED) # Cross Tab Frequency - Two Way Chi Square

In [1]:
from tabulate import tabulate
import os, saspy, itertools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
today = datetime.today().strftime("%Y-%m-%d")
now = datetime.now()

#Stats Import
from scipy.stats import wilcoxon 
from scipy.stats import shapiro 
from scipy.stats import kstest
# from numpy.random import seed
# from numpy.random import math

import pingouin as pg
from pingouin import pairwise_ttests, pairwise_tukey, pairwise_gameshowell
import statsmodels.stats.multicomp as mc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
import scipy.stats as stats
# from scipy import stats # Might need as is for ttest proportiality z or tdist
import statsmodels.api as sm
from bioinfokit.analys import stat
from statsmodels.formula.api import ols
from scipy.stats import wilcoxon
from statsmodels.stats.proportion import proportions_ztest



import io
from io import BytesIO
import base64


from operator import concat

# sys.path.append(os.path.join(sys.path[0],'C:\Junk\lamb_consulting'))
sys.path.append(os.path.join(sys.path[0],'U:\lamb_consulting'))


# import cox, means, regressions

from lamb_consulting import *

from lamb_consulting.means import means_desc, means_desc_class, aov
from lamb_consulting.freqs import freqs, chi_contingency, chi_oneway
from lamb_consulting.export import export_to_Excel
# from lamb_consulting.regressions import ols, regression_plot
from lamb_consulting.cox import cox
from lamb_consulting.kaplan_meier import km,km_plots


def freqme(ds,var1):
    f=ds[var1].value_counts(dropna=False)
    p=ds[var1].value_counts(normalize=True)
    df=pd.concat({f,p}, axis=1, keys=['frequency', 'percent'])
    df["cumfrequency"] = df["frequency"].cumsum()
    df["cumpercent"] = df["percent"].cumsum()
    return df



# For figure image to base 64 writing

def fig_to_base64(fig):
    img = io.BytesIO()
    fig.savefig(img, format='png',
                bbox_inches='tight')
    img.seek(0)

    return base64.b64encode(img.getvalue())

C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\outdated\utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [2]:
print(os.getcwd())
# tpath = "C:/Junk/Weisse/EHPSS/"
tpath = "U:/Fox/Weisse/EHPSS/"
fn="EHPSS20221207"

extension='.xlsx'
file = tpath+fn+extension

df_in = pd.read_excel((file), sheet_name='Data')

# Filter out age lt 90d 
# df = df_in.loc[(df_in['age_d']>=90)]
# Treating lt 90 as a class
df = df_in.copy()
df['contrb_v'].fillna(method='ffill')
df['wt_kg'].fillna(method='ffill')

df_90_730 = df.loc[(df['age_d'] >= 90) & (df['age_d'] <= 730) ]
df_730_1095 = df.loc[(df['age_d'] > 730) & (df['age_d'] <= 1095)]
df_1095 = df.loc[(df['age_d' ] > 1095)]

# 50%        733.000000
def age_lt_90(row):
    if row["age_d"] <= 90:
        return 1
    elif row["age_d"] > 90:
        return 0
    else:
        return -1

def age_lt_730(row):
    if row["age_d"] <= 730:
        return 1
    elif row["age_d"] > 730:
        return 0
    else:
        return -1

# Note changed 1 to second condition for greater than given variable name change
def age_gt_1095(row):
    if row["age_d"] <= 1095:
        return 0
    elif row["age_d"] > 1095:
        return 1
    else:
        return -1


def age_90_730(row):
    if row["age_d"] <= 90:
        return 1
    elif row["age_d"] <= 730:
        return 2
    elif row["age_d"] > 730:
        return 3
    else:
        return -1

def age_730_1095(row):
    if row["age_d"] <= 730:
        return 1
    elif row["age_d"] <= 1095:
        return 2
    elif row["age_d"] > 1095:
        return 3
    else:
        return -1


def male_n(row):
    if row["gender"] == 1 or row["gender"] == 2:
        return 1
    elif row["gender"] == 3 or row["gender"] == 4:
        return 0
    else:
        return -1


def male_c(row):
    if row["gender_1"] == 'MC' or row["gender_1"] == 'MI' or row["gender_1"] == 'NM' or row["gender_1"] == 'MN' or row["gender_1"] == 'ME' or row["gender_1"] == 'M':
        return "M"
    elif row["gender_1"] == 'FS' or row["gender_1"] == 'FI' or row["gender_1"] == 'FE': 
        return "F"
    else:
        return "error"

# Will revisit this to loop datafrmes into apply statements, giving error as being passed in as a string
my_dfs = ['df_90_730', 'df_730_1095', 'df_1095']

df['male_n'] = df.apply(male_n,axis=1)
df['male_c'] = df.apply(male_c,axis=1)
df['age_lt_90'] = df.apply(age_lt_90,axis=1)
df['age_lt_730'] = df.apply(age_lt_730,axis=1)
df['age_gt_1095'] = df.apply(age_gt_1095,axis=1)
df['age_90_730'] = df.apply(age_90_730,axis=1)
df['age_730_1095'] = df.apply(age_730_1095,axis=1)
    


df_90_730['male_n'] = df_90_730.apply(male_n,axis=1)
df_90_730['male_c'] = df_90_730.apply(male_c,axis=1)
df_90_730['age_lt_90'] = df_90_730.apply(age_lt_90,axis=1)
df_90_730['age_lt_730'] = df_90_730.apply(age_lt_730,axis=1)
df_90_730['age_gt_1095'] = df_90_730.apply(age_gt_1095,axis=1)
df_90_730['age_90_730'] = df_90_730.apply(age_90_730,axis=1)
df_90_730['age_730_1095'] = df_90_730.apply(age_730_1095,axis=1)




df_730_1095['male_n'] = df_730_1095.apply(male_n,axis=1)
df_730_1095['male_c'] = df_730_1095.apply(male_c,axis=1)
df_730_1095['age_lt_90'] = df_730_1095.apply(age_lt_90,axis=1)
df_730_1095['age_lt_730'] = df_730_1095.apply(age_lt_730,axis=1)
df_730_1095['age_gt_1095'] = df_730_1095.apply(age_gt_1095,axis=1)
df_730_1095['age_90_730'] = df_730_1095.apply(age_90_730,axis=1)
df_730_1095['age_730_1095'] = df_730_1095.apply(age_730_1095,axis=1)

df_1095['male_n'] = df_1095.apply(male_n,axis=1)
df_1095['male_c'] = df_1095.apply(male_c,axis=1)
df_1095['age_lt_90'] = df_1095.apply(age_lt_90,axis=1)
df_1095['age_lt_730'] = df_1095.apply(age_lt_730,axis=1)
df_1095['age_gt_1095'] = df_1095.apply(age_gt_1095,axis=1)
df_1095['age_90_730'] = df_1095.apply(age_90_730,axis=1)
df_1095['age_730_1095'] = df_1095.apply(age_730_1095,axis=1)



cont = ['age_d', 'wt_kg', 'pps_n']
cont_noage = ['wt_kg', 'pps_n']
cat = ['male_n', 'breed_n', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country', 'pps_n']
# cat_nopps = ['male_n', 'breed_n', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country']
cat_nopps = ['male_n', 'breed_n', 'shunt_ty', 'portal_v', 'system_v', 'contrb_v', 'hospital', 'country','age_lt_90','age_lt_730','age_gt_1095','age_90_730','age_730_1095']

# cont_describe = "age_d wt_kg pps_n"
# cat_describe = "male_c breed_nc shunt_ty portal_v system_v contrb_v hospital country pps_n"


# print(df["gender_1"].value_counts())
# print(df["gender"].value_counts())

# print(df["male_c"].value_counts())
# print(df["male_n"].value_counts())

print(df.shape)
print(df_90_730.shape)
print(df_730_1095.shape)
print(df_1095.shape)

u:\Fox\Weisse\EHPSS


C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Ken\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\loc

(1082, 33)
(520, 33)
(123, 33)
(417, 33)


# One Way Frequency

In [3]:

my_list = cat.copy()
my_list.remove("breed_n")
my_list += ['breed_n']

my_list += ['age_lt_90','age_lt_730','age_gt_1095','age_90_730','age_730_1095']



my_cat_df_list = ['cat_df', 'cat_df_90_730', 'cat_df_730_1095', 'cat_df_1095']
cat_df = pd.DataFrame()
cat_df_90_730 = pd.DataFrame()
cat_df_730_1095 = pd.DataFrame()
cat_df_1095 = pd.DataFrame()


for i in my_list:
    cat_df = cat_df.append(chi_oneway(df,i,"df"))

for i in my_list:
    cat_df_90_730 = cat_df_90_730.append(chi_oneway(df_90_730,i,"df_90_730"))

for i in my_list:
    cat_df_730_1095 = cat_df_730_1095.append(chi_oneway(df_730_1095,i,"df_730_1095"))

for i in my_list:
    cat_df_1095 = cat_df_1095.append(chi_oneway(df_1095,i,"df_1095"))

print(cat_df.shape)
print(cat_df.head(2))
print(cat_df_90_730.shape)
print(cat_df_90_730.head(2))
print(cat_df_730_1095.shape)
print(cat_df_730_1095.head(2))
print(cat_df_1095.shape)
print(cat_df_1095.head(2))








(176, 8)
  df_used         table  var_level  frequency   percent  cumfrequency  \
0      df  Table male_n        0.0        575  0.531423           575   
1      df  Table male_n        1.0        507  0.468577          1082   

   cumpercent  p_value  
0    0.531423  0.03871  
1    1.000000  0.03871  
(139, 8)
     df_used         table  var_level  frequency   percent  cumfrequency  \
0  df_90_730  Table male_n        0.0        265  0.509615           265   
1  df_90_730  Table male_n        1.0        255  0.490385           520   

   cumpercent   p_value  
0    0.509615  0.661003  
1    1.000000  0.661003  
(91, 8)
       df_used         table  var_level  frequency   percent  cumfrequency  \
0  df_730_1095  Table male_n        0.0         80  0.650407            80   
1  df_730_1095  Table male_n        1.0         43  0.349593           123   

   cumpercent   p_value  
0    0.650407  0.000849  
1    1.000000  0.000849  
(120, 8)
   df_used         table  var_level  frequency   p

# Create Top 5 Breed dataset based of of the one-way Chi-Squre Output

In [4]:
# breed_t5_list = [62, 61, 53, 38, 35]
breed_t5_list = df['breed_n'].value_counts().keys()[0:5].sort_values(ascending=False).tolist()
breed_mask = (df['breed_n'].isin(breed_t5_list)) 
breed_t5 = df.loc[breed_mask]
print(breed_t5_list)
print(df.shape)
print(breed_t5.shape)

[62.0, 61.0, 53.0, 38.0, 35.0]
(1082, 33)
(648, 33)


# Two Way Frequency

In [5]:
my_list = cat.copy()
# my_list += ['age_lt_730']
my_list += ['age_lt_90','age_lt_730','age_gt_1095','age_90_730','age_730_1095']

iter_list = list(itertools.combinations(my_list, 2))
iter_list

# my_ct_cat_df_names = ['cross_cat', 'cross_cat_90_730', 'cross_cat_730_1095', 'cross_cat_1095']
# my_ct_cat_df_list = [pd.DataFrame() for i in my_ct_cat_df_names]
# my_ct_cat_df = dict(zip(my_ct_cat_df_names, my_ct_cat_df_list))

# cross_cat_90_730 = pd.DataFrame()
# cross_cat_730_1095 = pd.DataFrame()
# cross_cat_1095 = pd.DataFrame()



print(len(iter_list))
cross_cat = pd.DataFrame()
cross_cat_df_90_730 = pd.DataFrame()
cross_cat_df_730_1095 = pd.DataFrame()
cross_cat_df_1095 = pd.DataFrame()

ct = 1

if ct ==1:
    f = open('df_ct.html','w')

my_len = (len(iter_list))
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    a = res[0] 
    v1,v2 = a
    cross_tmp, my_df = chi_contingency(df,v1,v2,"1","chi_cont","df")
    cross_cat = cross_cat.append(cross_tmp)
    if ct ==1:
        my_html = my_df.to_html()
        f.write("The Cross Tabular variables are {} and {}.".format(v1, v2))
        f.write(my_html)

if ct ==1:
    f.close()


# df_90_730
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    a = res[0] 
    v1,v2 = a
    cross_tmp, my_df = chi_contingency(df_90_730,v1,v2,"1","chi_cont","df_90_730")
    cross_cat_df_90_730 = cross_cat_df_90_730.append(cross_tmp)


# df_730_1095
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    a = res[0] 
    v1,v2 = a
    cross_tmp, my_df = chi_contingency(df_730_1095,v1,v2,"1","chi_cont","df_730_1095")
    cross_cat_df_730_1095 = cross_cat_df_730_1095.append(cross_tmp)


# df_1095
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    a = res[0] 
    v1,v2 = a
    cross_tmp, my_df = chi_contingency(df_1095,v1,v2,"1","chi_cont","df_1095")
    cross_cat_df_1095 = cross_cat_df_1095.append(cross_tmp)

print("cross_cat")
print(cross_cat.tail(5))
print("cross_cat_df_90_730")
print(cross_cat_df_90_730.tail(5))
print("cross_cat_df_730_1095")
print(cross_cat_df_730_1095.tail(5))
print("cross_cat_df_1095")
print(cross_cat_df_1095.tail(5))

91
cross_cat
  df_used r_q_n  analysis research_q                            Table  \
5      df     1  chi_cont          1  Table age_90_730 * age_730_1095   
6      df     1  chi_cont          1  Table age_90_730 * age_730_1095   
7      df     1  chi_cont          1  Table age_90_730 * age_730_1095   
8      df     1  chi_cont          1  Table age_90_730 * age_730_1095   
9      df     1  chi_cont          1  Table age_90_730 * age_730_1095   

   Frequency     Percent Missing iv1 iv2  
5          0    0.000000       .   2   3  
6          0    0.000000       .   3   1  
7        123   11.367837       .   3   2  
8        417   38.539741       .   3   3  
9       1082  100.000000       0   .   .  
cross_cat_df_90_730
     df_used r_q_n  analysis research_q                             Table  \
1  df_90_730     1  chi_cont          1    Table age_gt_1095 * age_90_730   
0  df_90_730     1  chi_cont          1  Table age_gt_1095 * age_730_1095   
1  df_90_730     1  chi_cont          1

# Descriptive Means first Overall then by Class

In [6]:


m_desc_df = pd.DataFrame()
m_desc_df_90_730 = pd.DataFrame()
m_desc_df_730_1095 = pd.DataFrame()
m_desc_df_1095 = pd.DataFrame()

m_class_df = pd.DataFrame()
m_class_df_90_730 = pd.DataFrame()
m_class_df_730_1095 = pd.DataFrame()
m_class_df_1095 = pd.DataFrame()

for i in cont:
        desc_tmp = means_desc(df, i,"df")
        m_desc_df = m_desc_df.append(desc_tmp)
# df_90_730
        desc_tmp = means_desc(df_90_730, i,"df_90_730")
        m_desc_df_90_730 = m_desc_df_90_730.append(desc_tmp)
# df_730_1095
        desc_tmp = means_desc(df_730_1095, i,"df_730_1095")
        m_desc_df_730_1095 = m_desc_df_730_1095.append(desc_tmp)
# df_1095
        desc_tmp = means_desc(df_1095, i,"df_1095")
        m_desc_df_1095 = m_desc_df_1095.append(desc_tmp)
print("m_desc_df")
print(m_desc_df)
print("m_desc_df_90_730")
print(m_desc_df_90_730)
print("m_desc_df_730_1095")
print(m_desc_df_730_1095)
print("m_desc_df_1095")
print(m_desc_df_1095)

for i in cat:
    for j in cont:
        desc_class_tmp = means_desc_class(df,j, i,"df")
        m_class_df = m_class_df.append(desc_class_tmp)
# df_90_730
        desc_class_tmp = means_desc_class(df_90_730,j, i,"df_90_730")
        m_class_df_90_730 = m_class_df_90_730.append(desc_class_tmp)
# df_730_1095
        desc_class_tmp = means_desc_class(df_730_1095,j, i,"df_730_1095")
        m_class_df_730_1095 = m_class_df_730_1095.append(desc_class_tmp)
# df_1095
        desc_class_tmp = means_desc_class(df_1095,j, i,"df_1095")
        m_class_df_1095 = m_class_df_1095.append(desc_class_tmp)

print("m_class_df")       
print(m_class_df)
print("m_class_df_90_730")
print(m_class_df_90_730)
print("m_class_df_730_1095")
print(m_class_df_730_1095)
print("m_class_df_1095")
print(m_class_df_1095)


m_desc_df
  df_used cont_var  nmiss       n         mean  median          std   min  \
0      df    age_d      0  1082.0  1427.213678  722.50  3668.610398  0.00   
0      df    wt_kg     15  1067.0     4.828744    4.44     2.627325  0.83   
0      df    pps_n      4  1078.0     3.336735    4.00     1.630935  1.00   

       max     p25      p75  
0  41319.0  298.00  1682.50  
0     12.0    2.61     6.35  
0      5.0    2.00     5.00  
m_desc_df_90_730
     df_used cont_var  nmiss      n        mean  median         std    min  \
0  df_90_730    age_d      0  520.0  341.446154  306.50  168.739875  93.00   
0  df_90_730    wt_kg      4  516.0    4.220756    3.65    2.505646   0.83   
0  df_90_730    pps_n      3  517.0    2.733075    2.00    1.697675   1.00   

     max     p25     p75  
0  728.0  200.00  453.25  
0   12.0    2.15    5.60  
0    5.0    1.00    5.00  
m_desc_df_730_1095
       df_used cont_var  nmiss      n        mean  median         std     min  \
0  df_730_1095    age_d

# Diffs with Tukeys Adjustment

In [7]:
# Set up affectionate naming for stratified LOB
# lob_dict_in = {1:['df']}
lob_dict_in = {1:['df'], 2:['df_90_730'], 3:['df_730_1095'], 4:['df_1095']}
lob_df = pd.DataFrame(lob_dict_in)
lob_df = lob_df.T.reset_index()
lob_df.rename(columns={0: 'ds_name', 'index': 'df_num'}, inplace = True)


# Set up affectionate Dict naming for DV
dv_dict_in = {1:['wt_kg'], 2:['age_d'], 3:['pps_n']}
dv_df = pd.DataFrame(dv_dict_in)
dv_df = dv_df.T.reset_index()
dv_df.rename(columns={0: 'dv_name', 'index':'dv_num'}, inplace=True)


# Set up Dict for Model ID to b merge from above dict
# lob_ds = {1:df}
lob_ds = {1:df, 2:df_90_730, 3:df_730_1095, 4:df_1095}


res = stat()
data_dict = {}
mt_data = pd.DataFrame()

# Set normality statistics
normal_on = 0
if normal_on != 0:
    f = open('my_normality.html','w')


# Set up dict for Full DVs Full Rank but Catgetoricals with No PPSN
dvs = {1:'wt_kg', 2:'age_d', 3:'pps_n'}
# Created new list with pps removed as IV for use as a DV
# for i in cat:
for i in cat_nopps:
    for k1,v1 in dvs.items():
        for k, v in lob_ds.items():
            anova_df = df.loc[:,[v1,i]].dropna()
            # res.tukey_hsd(df=v, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')
            res.tukey_hsd(df=anova_df, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')
            #Normality Section
            if normal_on !=0:
                my_shapiro = shapiro(res.anova_std_residuals)
                my_shapiro = str(my_shapiro)
                my_ks = kstest(res.anova_std_residuals, 'norm')
                my_ks = str(my_ks)

                # residuals qq
                sm.qqplot(res.anova_std_residuals,line='45')
                plt.xlabel('Theoretic Quantiles '+ v1+ ' ' + i)
                plt.ylabel("Standardized Residuals")
                my_fig0 = plt
                encoded0 = fig_to_base64(my_fig0)
                my_html0 = '<img src="data:image/png;base64, {}">'.format(encoded0.decode('utf-8'))
                f.write("The residuals normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html0)

                # residuals histogram
                my_fig1 = plt.figure()
                plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
                plt.xlabel('Residuals' + ' ' + i + ' ' + v1)
                plt.ylabel('Frequency')
                plt.show()

                # Histogram Writing
                tmpfile = BytesIO()
                my_fig1.savefig(tmpfile, format='png')            
                encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')
                # encoded = base64.b64encode(tmpfile.getvalue()).decode('utf8') <img src='data:image/png;base64,{{}}'>
                my_html1 = '---' + '<img src=\'data:image/png;base64,{}\'>'.format(encoded) + '---'
                f.write("The histogram normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html1)



            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            mt_data = mt_data.append(data_dict[k])



# Get PPN as Sole IV with Wt and Age as DV
# Had to remove pps_n as a DV since it was an IV with dropna analysis
dvs = {1:'wt_kg', 2:'age_d'}
# for i in cat:
cat_pps_n = ['pps_n']
for i in cat_pps_n:
    for k1,v1 in dvs.items():
        for k, v in lob_ds.items():
            anova_df = df.loc[:,[v1,i]].dropna()
            res.tukey_hsd(df=anova_df, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')

            #Normality Section
            if normal_on !=0:
                my_shapiro = shapiro(res.anova_std_residuals)
                my_shapiro = str(my_shapiro)
                my_ks = kstest(res.anova_std_residuals, 'norm')
                my_ks = str(my_ks)
                # residuals qq
                sm.qqplot(res.anova_std_residuals,line='45')
                plt.xlabel('Theoretic Quantiles '+ v1+ ' ' + i)
                plt.ylabel("Standardized Residuals")
                plt.show()

                my_fig3 = plt
                encoded3 = fig_to_base64(my_fig3)
                my_html3 = '<img src="data:image/png;base64, {}">'.format(encoded3.decode('utf-8'))
                f.write("The residuals normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html3)

                # residuals histogram
                my_fig4 = plt.figure()
                plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
                plt.xlabel('Residuals' + ' ' + i + ' ' + v1)
                plt.ylabel('Frequency')
                plt.show()

                # Histogram Writing
                tmpfile = BytesIO()
                my_fig4.savefig(tmpfile, format='png')            
                encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')
                my_html4 = '---' + '<img src=\'data:image/png;base64,{}\'>'.format(encoded) + '---'
                f.write("The histogram normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html4)


            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            mt_data = mt_data.append(data_dict[k])




# Get Age LT 730 as Sole IV with Wt and wt and PPS
# Had to remove pps_n as a DV since it was an IV with dropna analysis
dvs = {1:'wt_kg', 2:'pps_n'}
# for i in cat:
cat_age_lt_730_n = ['age_lt_730']
for i in cat_age_lt_730_n:
    for k1,v1 in dvs.items():
        for k, v in lob_ds.items():
            anova_df = df.loc[:,[v1,i]].dropna()
            res.tukey_hsd(df=anova_df, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')

            #Normality Section
            if normal_on !=0:
                my_shapiro = shapiro(res.anova_std_residuals)
                my_shapiro = str(my_shapiro)
                my_ks = kstest(res.anova_std_residuals, 'norm')
                my_ks = str(my_ks)
                # residuals qq
                sm.qqplot(res.anova_std_residuals,line='45')
                plt.xlabel('Theoretic Quantiles '+ v1+ ' ' + i)
                plt.ylabel("Standardized Residuals")
                plt.show()

                my_fig5 = plt
                encoded5 = fig_to_base64(my_fig5)
                my_html5 = '<img src="data:image/png;base64, {}">'.format(encoded5.decode('utf-8'))
                f.write("The residuals normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html5)

                # residuals histogram
                my_fig6 = plt.figure()
                plt.hist(res.anova_model_out.resid, bins='auto', histtype='bar', ec='k') 
                plt.xlabel('Residuals' + ' ' + i + ' ' + v1)
                plt.ylabel('Frequency')
                plt.show()

                # Histogram Writing
                tmpfile = BytesIO()
                my_fig6.savefig(tmpfile, format='png')            
                encoded = base64.b64encode(tmpfile.getvalue()).decode('utf-8')
                my_html6 = '---' + '<img src=\'data:image/png;base64,{}\'>'.format(encoded) + '---'
                f.write("The histogram normality plot is based upon {} and {} and KS is {} and Shapiro is {}.".format(v1, i, my_ks, my_shapiro))
                f.write(my_html6)


            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            mt_data = mt_data.append(data_dict[k])



diffs = pd.merge(mt_data, lob_df, on=["df_num"])
diffs = pd.merge(diffs, dv_df, on=["dv_num"])
diffs

if normal_on != 0:
    f.close()


# ANOVA of continuous vars with top 5 Breed IV

In [8]:

res = stat()
data_dict = {}
t5_breed_data = pd.DataFrame()

dvs = {1:'wt_kg', 2:'age_d', 3:'pps_n'}

cat_breed = ['breed_n']
ds_breed = {1:breed_t5}

for i in cat_breed:
    for k1,v1 in dvs.items():
        for k, v in ds_breed.items():
            res.tukey_hsd(df=v, res_var=v1, xfac_var=i, anova_model=v1 + ' ~ C(' + i + ')')
            tmp = res.tukey_summary
            tmp['df_num'] = k
            tmp['dv_num'] = k1
            tmp['iv'] = i
            data_dict[k] = tmp
            t5_breed_data = t5_breed_data.append(data_dict[k])


breed_diffs = pd.merge(t5_breed_data, lob_df, on=["df_num"])
breed_diffs = pd.merge(breed_diffs, dv_df, on=["dv_num"])
breed_diffs

,group1,group2,Diff,Lower,Upper,q-value,p-value,df_num,dv_num,iv,ds_name,dv_name
0,53.0,35.0,2.312477,1.257804,3.367149,8.526555,0.001000,1,1,breed_n,df,wt_kg
1,53.0,62.0,0.150122,-1.254606,1.554849,0.415590,0.900000,1,1,breed_n,df,wt_kg
2,53.0,61.0,2.537859,1.525402,3.550316,9.747758,0.001000,1,1,breed_n,df,wt_kg
3,53.0,38.0,0.102148,-1.066836,1.271132,0.339809,0.900000,1,1,breed_n,df,wt_kg
4,35.0,62.0,2.462598,1.158432,3.766765,7.343016,0.001000,1,1,breed_n,df,wt_kg
5,35.0,61.0,0.225382,-0.642178,1.092943,1.010261,0.900000,1,1,breed_n,df,wt_kg
6,35.0,38.0,2.414625,1.368625,3.460624,8.977021,0.001000,1,1,breed_n,df,wt_kg
7,62.0,61.0,2.687981,1.417711,3.958250,8.228943,0.001000,1,1,breed_n,df,wt_kg
8,62.0,38.0,0.047974,-1.350253,1.446201,0.133426,0.900000,1,1,breed_n,df,wt_kg
9,61.0,38.0,2.640007,1.636588,3.643426,10.231440,0.001000,1,1,breed_n,df,wt_kg


# Write to Excel

In [9]:
# my_sheets = ['cat_one' 'cross_cat' 'm_desc' 'm_class' 'diffs' 'breed_diffs']

writer = pd.ExcelWriter('ephss_out_20230107.xlsx', engine = 'xlsxwriter')
# for i in my_sheets:
#     [i].to_excel(writer,sheet_name=i)

# Data In
df.to_excel(writer,sheet_name='df')
df_90_730.to_excel(writer,sheet_name='df_90_730')
df_730_1095.to_excel(writer,sheet_name='df_730_1095')
df_1095.to_excel(writer,sheet_name='df_1095')
# One Way Freq
cat_df.to_excel(writer,sheet_name='cat_df')
cat_df_90_730.to_excel(writer,sheet_name='cat_df_90_730')
cat_df_730_1095.to_excel(writer,sheet_name='cat_df_730_1095')
cat_df_1095.to_excel(writer,sheet_name='cat_df_1095')
# Cross Tab Frequency
cross_cat.to_excel(writer,sheet_name='cross_cat')
cross_cat_df_90_730.to_excel(writer,sheet_name='cross_cat_df_90_730')
cross_cat_df_730_1095.to_excel(writer,sheet_name='cross_cat_df_730_1095')
cross_cat_df_1095.to_excel(writer,sheet_name='cross_cat_df_1095')
# Means overall
m_desc_df.to_excel(writer,sheet_name='m_desc_df')
m_desc_df_90_730.to_excel(writer,sheet_name='m_desc_df_90_730')
m_desc_df_730_1095.to_excel(writer,sheet_name='m_desc_df_730_1095')
m_desc_df_1095.to_excel(writer,sheet_name='m_desc_df_1095')
# Means with class
m_class_df.to_excel(writer,sheet_name='m_class_df')
m_class_df_90_730.to_excel(writer,sheet_name='m_class_df_90_730')
m_class_df_730_1095.to_excel(writer,sheet_name='m_class_df_730_1095')
m_class_df_1095.to_excel(writer,sheet_name='m_class_df_1095')
# Diffs
diffs.to_excel(writer,sheet_name='diffs')
breed_diffs.to_excel(writer,sheet_name='breed_diffs')
writer.save()


# Single df write
# my_sheets = ['df']
# writer = pd.ExcelWriter('df.xlsx', engine = 'xlsxwriter')
# df.to_excel(writer,sheet_name='data')
# writer.save()


In [29]:
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

# can we assume anything from our sample
significance = 0.025

def my_z_test(denom_in, num_in, denom_grp_1, denom_grp_2, grp_1, grp_2):
    proportion_data = pd.DataFrame()
    # Step 1 country is in index
    tmp = pd.crosstab(df[denom_in].fillna('.'),df[num_in].fillna('.'))
    tmp = tmp.drop(columns=['.'])
    tmp.reset_index(inplace=True)
    # Work on values

    tmp_1a = tmp.loc[tmp[denom_in] == denom_grp_1] # Country is 0
    tmp_1a = tmp_1a.drop([denom_in], axis=1)
    denom_1 = tmp_1a.loc[denom_grp_1].sum() # Country is 0
    # Get numerator from single row
    tmp_1b = tmp_1a[[grp_1]].reset_index() # pps_n is 1
    tmp_1b = tmp_1b.drop(['index'], axis=1)
    # Get only column and sum, if no sum will take full dataframe and not just numerator
    num_1 = tmp_1b.loc[0].sum()
    tup1 = (num_1, denom_1)
    # print("tup1")
    # print(tup1)
    pct_1 = num_1 / denom_1


    # Get single row to be analyzed for upcoming comparision
    tmp_2a = tmp.loc[tmp[denom_in] == denom_grp_2]
    tmp_2a = tmp_2a.drop([denom_in], axis=1)
    denom_2 = tmp_2a.loc[denom_grp_2].sum() # Country 1
    # Get numerator from single row e.g. pps_n 1 is first column, 2 is second column
    tmp_2b = tmp_2a[[grp_2]].reset_index() # pps_n is 1
    tmp_2b = tmp_2b.drop(['index'], axis=1)
    num_2 = tmp_2b.loc[0].sum() 
    tup2 = (num_2, denom_2)
    # print("tup2")
    # print(tup2)
    pct_2 = num_2 / denom_2
    sample_success_a, sample_size_a = tup1
    sample_success_b, sample_size_b = tup2
    successes = np.array([sample_success_a, sample_success_b])
    samples = np.array([sample_size_a, sample_size_b])

    # note, no need for a Ho value here - it's derived from the other parameters
    stat, p_value = proportions_ztest(count=successes, nobs=samples, alternative='two-sided')


    # narrative = ('Comparing: Pct 1: %0.3f, Pct 2: %0.3f' % (pct_1, pct_2) + " Var 1: " + denom_in + " Var 2: " + num_in)
    narrative = [('Comparing: Pct 1: %0.3f, Pct 2: %0.3f Num 1: %0.0f, Denom 1: %0.0f, Num 2: %0.0f, Denom 2: %0.0f'% (pct_1, pct_2, num_1, denom_1, num_2, denom_2) + 
    ", Var 1: " + denom_in + ', Denom Levels: %0.0f/%0.0f' % (denom_grp_1, denom_grp_2) + ', Num Levels: %0.0f/%0.0f' % (grp_1, grp_2) + ', P-Value is: %0.3f' % (p_value))]
    #  ", Var 1: " + denom_in + " " + denom_grp_1 + "/" + denom_grp_2 + ", Var 2: " + num_in + grp_1 + "/" + grp_2 + ', P-Value is: %0.3f' % (p_value))]
    #  "/" + denom_grp_2 + ", Var 2: " + num_in + grp_1 + "/" + grp_2 + 
    print(narrative)
    tmp_df = pd.DataFrame({'narrative' : narrative, 'p_value' : p_value})

    proportion_data = pd.concat([my_df,tmp_df])
    # proportion_data = proportion_data.append(tmp_df)

    # return narrative, p_value
    return proportion_data


my_df = pd.DataFrame()

# The first two are comparing pairwise for the first variable and should be different e.g. country 0/1, 0,2, 0/3
# The last two should be the same since we are comparing the same level of the second variable within the pairwise first variable e.g. country 0 pps 1 vs. country 1 pps 1

# Country 0 vs 1, 2, 3 | PPS 1
my_df = my_z_test('country', 'pps_n', 0, 1, 1, 1) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 1), col val 1 (pps 1), col val 2 (pps 1)
my_df = my_z_test('country', 'pps_n', 0, 2, 1, 1) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 2), col val 1 (pps 1), col val 2 (pps 1)
my_df = my_z_test('country', 'pps_n', 0, 3, 1, 1) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 3), col val 1 (pps 1), col val 2 (pps 1)
# Country 0 vs 1, 2, 3 | PPS 2
my_df = my_z_test('country', 'pps_n', 0, 1, 2, 2) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 1), col val 1 (pps 2), col val 2 (pps 2)
my_df = my_z_test('country', 'pps_n', 0, 2, 2, 2) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 2), col val 1 (pps 2), col val 2 (pps 2)
my_df = my_z_test('country', 'pps_n', 0, 3, 2, 2) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 3), col val 1 (pps 2), col val 2 (pps 2)
# Country 0 vs 1, 2, 3 | PPS 3
my_df = my_z_test('country', 'pps_n', 0, 1, 3, 2) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 1), col val 1 (pps 3), col val 2 (pps 3)
my_df = my_z_test('country', 'pps_n', 0, 2, 3, 2) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 2), col val 1 (pps 3), col val 2 (pps 3)
my_df = my_z_test('country', 'pps_n', 0, 3, 3, 2) # row var (country), col var (pps_n), row val 1 (country 0), row val 2 (country 3), col val 1 (pps 3), col val 2 (pps 3)
# Country 1 vs 2, 3 | PPS 1 and 2 and 3
my_df = my_z_test('country', 'pps_n', 1, 2, 1, 1)
my_df = my_z_test('country', 'pps_n', 1, 3, 1, 1)
my_df = my_z_test('country', 'pps_n', 1, 2, 2, 2)
my_df = my_z_test('country', 'pps_n', 1, 3, 2, 2)
my_df = my_z_test('country', 'pps_n', 1, 2, 3, 3)
my_df = my_z_test('country', 'pps_n', 1, 3, 3, 3)
# Country 0 vs 1, 2, 3 | PPS 1 and 2 and 3
my_df = my_z_test('country', 'pps_n', 2, 3, 1, 1)
my_df = my_z_test('country', 'pps_n', 2, 3, 2, 2)
my_df = my_z_test('country', 'pps_n', 2, 3, 3, 3)

['Comparing: Pct 1: 0.271, Pct 2: 0.258 Num 1: 220, Denom 1: 812, Num 2: 16, Denom 2: 62, Var 1: country, Denom Levels: 0/1, Num Levels: 1/1, P-Value is: 0.826']
['Comparing: Pct 1: 0.271, Pct 2: 0.292 Num 1: 220, Denom 1: 812, Num 2: 14, Denom 2: 48, Var 1: country, Denom Levels: 0/2, Num Levels: 1/1, P-Value is: 0.754']
['Comparing: Pct 1: 0.271, Pct 2: 0.122 Num 1: 220, Denom 1: 812, Num 2: 19, Denom 2: 156, Var 1: country, Denom Levels: 0/3, Num Levels: 1/1, P-Value is: 0.000']
['Comparing: Pct 1: 0.087, Pct 2: 0.097 Num 1: 71, Denom 1: 812, Num 2: 6, Denom 2: 62, Var 1: country, Denom Levels: 0/1, Num Levels: 2/2, P-Value is: 0.803']
['Comparing: Pct 1: 0.087, Pct 2: 0.146 Num 1: 71, Denom 1: 812, Num 2: 7, Denom 2: 48, Var 1: country, Denom Levels: 0/2, Num Levels: 2/2, P-Value is: 0.171']
['Comparing: Pct 1: 0.087, Pct 2: 0.051 Num 1: 71, Denom 1: 812, Num 2: 8, Denom 2: 156, Var 1: country, Denom Levels: 0/3, Num Levels: 2/2, P-Value is: 0.131']
['Comparing: Pct 1: 0.123, Pct 2

# Prob TTDIST from Excel Proportionality

In [16]:
# From Excel Example
# https://stackoverflow.com/questions/56991534/excels-tdist-in-scipy
# from scipy import stats
stats.t.sf(.66299299354, df=9999999999999) * 2


0.5073350414125459

In [ ]:
# https://github.com/sonalake/blog-hypothesis-testing/blob/master/examples/proportions/2-sample-z-test.py
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

# can we assume anything from our sample
significance = 0.025

# our samples;  note - the samples do not need to be the same size

# my_a = (16, 62)
# my_b = (220, 812)
# sample_success_a, sample_size_a = my_a
# sample_success_b, sample_size_b = my_b

# here 410 samples out of 500 were "ok"
sample_success_a, sample_size_a = (16, 62)
# here 379 samples out of 400 were "ok"
sample_success_b, sample_size_b = (220, 812)

# check our sample against Ho for Ha != Ho
successes = np.array([sample_success_a, sample_success_b])
samples = np.array([sample_size_a, sample_size_b])

# note, no need for a Ho value here - it's derived from the other parameters
stat, p_value = proportions_ztest(count=successes, nobs=samples, alternative='two-sided')

# report
print('z_stat: %0.3f, p_value: %0.3f' % (stat, p_value))

if p_value > significance:
    print("Fail to reject the null hypothesis - we have nothing else to say")
else:
    print("Reject the null hypothesis - suggest the alternative hypothesis is true")

z_stat: -0.220, p_value: 0.826
Fail to reject the null hypothesis - we have nothing else to say


In [ ]:
def str2frame(estr, sep = ',', lineterm = '\n', set_header = True):
    dat = [x.split(sep) for x in estr.split(lineterm)][1:-1]
    df = pd.DataFrame(dat)
    if set_header:
        df = df.T.set_index(0, drop = True).T # flip, set ix, flip back
    return df

In [ ]:
# Step 1 country is in index
tmp = pd.crosstab(df['country'].fillna('.'),df['pps_n'].fillna('.'))
tmp = tmp.drop(columns=['.'])
tmp.reset_index(inplace=True)
# Get country out of index
# print("tmp_a2")
# print(tmp_a.head(10))


# Get single row to be analyzed for upcoming comparision
tmp_1a = tmp.loc[tmp['country'] == 0]
tmp_1a = tmp_1a.drop(['country'], axis=1)
# print("tmp_b")
# print(tmp_b.head(10))
denom_1 = tmp_1a.loc[0].sum() # Country 0
# print("denom_0")
# print(denom_0)
# type(denom_0)

# Get numerator from single row
tmp_1b = tmp_1a[[1]].reset_index() # pps_n is 1
tmp_1b = tmp_1b.drop(['index'], axis=1)
# print("tmp_c")
# print(tmp_c.head(10))
# Get only column and sum, if no sum will take full dataframe and not just numerator
num_1 = tmp_1b.loc[0].sum() 
# print("num_0")
# print(num_0)
# print(type(num_0))
tup1 = (num_1, denom_1)
print("tup1")
print(tup1)
type(tup1)


# Get single row to be analyzed for upcoming comparision
tmp_2a = tmp.loc[tmp['country'] == 1]
tmp_2a = tmp_2a.drop(['country'], axis=1)
# print("tmp_2a")
# print(tmp_2a.head(10))
denom_2 = tmp_2a.loc[1].sum() # Country 1
print("denom_2")
print(denom_2)
# Get numerator from single row e.g. pps_n 1 is first column, 2 is second column
tmp_2b = tmp_2a[[1]].reset_index() # pps_n is 1
tmp_2b = tmp_2b.drop(['index'], axis=1)
num_2 = tmp_2b.loc[0].sum() 
tup2 = (num_2, denom_2)
print("tup2")
print(tup2)
type(tup2)



sample_success_a, sample_size_a = tup1
# here 379 samples out of 400 were "ok"
sample_success_b, sample_size_b = tup2

# check our sample against Ho for Ha != Ho
successes = np.array([sample_success_a, sample_success_b])
samples = np.array([sample_size_a, sample_size_b])

# note, no need for a Ho value here - it's derived from the other parameters
stat, p_value = proportions_ztest(count=successes, nobs=samples, alternative='two-sided')

# report
print('z_stat: %0.3f, p_value: %0.3f' % (stat, p_value))

if p_value > significance:
    print("Fail to reject the null hypothesis - we have nothing else to say")
else:
    print("Reject the null hypothesis - suggest the alternative hypothesis is true")

tup1
(220, 812)
denom_2
62
tup2
(16, 62)
z_stat: 0.220, p_value: 0.826
Fail to reject the null hypothesis - we have nothing else to say
